In [27]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

## 非线性0-1规划

In [28]:
m_QP = gp.Model("非线性")

In [29]:
x = m_QP.addVars(3, name="x", vtype=GRB.BINARY)

In [30]:
m_QP.setObjective(x[0] + x[0] * x[1] - x[2], GRB.MAXIMIZE)

In [31]:
m_QP.addConstr(-2 * x[0] + 3 * x[1] + x[2] <= 3)

<gurobi.Constr *Awaiting Model Update*>

In [32]:
m_QP.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0x57799563
Model has 1 quadratic objective term
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 1.0000000
Presolve removed 1 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 3: 2 1 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e

In [33]:
print(f"Objective value: {m_QP.ObjVal}")
print("Optimal solution:")
for i in m_QP.getVars():
    if i.X > 1e-6:
        print(f"{i.VarName} = {i.X}", end="\n")

Objective value: 2.0
Optimal solution:
x[0] = 1.0
x[1] = 1.0


# 线性化

做变量替换$y=x_1x_2$，有：
$$x_1+x_2-1\le y\le x_1$$
$$x_1+x_2-1\le y\le x_2$$
从而有线性规划：

In [34]:
m_LP = gp.Model("线性")

In [35]:
x = m_LP.addVars(3, vtype=GRB.BINARY, name="x")

In [36]:
y = m_LP.addVar(vtype=GRB.BINARY, name="y")

In [37]:
m_LP.setObjective(x[0] + y - x[2], sense=GRB.MAXIMIZE)

In [38]:
m_LP.addConstr(-2 * x[0] + 3 * x[1] + x[2] <= 3)
m_LP.addConstr(x[0] + x[1] - 1 <= y)
m_LP.addConstr(y <= x[0])
m_LP.addConstr(x[0] + x[1] - 1 <= y)
m_LP.addConstr(y <= x[1])

<gurobi.Constr *Awaiting Model Update*>

In [39]:
m_LP.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 4 columns and 13 nonzeros
Model fingerprint: 0xf5d0a996
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 1.0000000
Found heuristic solution: objective 2.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 2 1 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%


In [40]:
print(f"Objective value: {m_LP.ObjVal}")
print("Optimal solution:")
for i in m_LP.getVars():
    if i.X > 1e-6:
        print(f"{i.VarName} = {i.X}", end="\n")

Objective value: 2.0
Optimal solution:
x[0] = 1.0
x[1] = 1.0
y = 1.0
